In [9]:
import csv
import itertools
import os

import requests

# Entity Service: Multiparty linkage demo
This notebook is a demonstration of the multiparty linkage capability that has been implemented in the Entity Service.

We show how five parties may upload their hashed data to the Entity Service to obtain a multiparty linkage result. This result identifies each entity across all datasets in which they are included.

## Check the status of the Entity Service
Ensure that it is running and that we have the correct version. Multiparty support was introduced in version 1.11.0.

In [6]:
PREFIX = "http://localhost:8851/api/v1"
print(requests.get(f"{PREFIX}/status").json())
print(requests.get(f"{PREFIX}/version").json())

{'project_count': 0, 'rate': 1, 'status': 'ok'}
{'anonlink': '0.11.2', 'entityservice': 'v1.11.0', 'python': '3.6.8'}


## Create a new project
We create a new multiparty project for five parties by specifying the number of parties and the output type (currently only the `group` output type supports multiparty linkage). Retain the `project_id`, so we can find the project later. Also retain the `result_token`, so we can retrieve the results (careful: anyone with this token has access to the results). Finally, the `update_tokens` identify the five data data providers and permit them to upload CLKs.

In [7]:
project_info = requests.post(
    f"{PREFIX}/projects",
    json={
        "schema": {},
        "result_type": "groups",
        "number_parties": 5,
        "name": "example project"
    }
).json()
project_id = project_info["project_id"]
result_token = project_info["result_token"]
update_tokens = project_info["update_tokens"]

print("project_id:", project_id)
print()
print("result_token:", result_token)
print()
print("update_tokens:", update_tokens)

project_id: bf373054ee0ed0cd5698dbc43b36c87744b134993ecb843d

result_token: 6684ab2f06587ca3079d02651c8265c1b35f3be8e8fe238e

update_tokens: ['d2c1084f091b7d31d2dc1d18b42570e78f8697bb912dfde0', 'a762cebc30701bf860c7186aec940e8b303f272ddb991398', 'dd079d56422dc0c295f6c93df5e25e2e91ba3d01cdd66aa4', '50d25349d8a4d3a9e81c55ab2cb92fc765f0091b035e60ff', '714af297548d14693a9ffb1e598567f9900a754c8b6c89e7']


## Upload the hashed data
This is where each party uploads their CLKs into the service. Here, we do the work of all five data providers inside this for loop. In a deployment scenario, each data provider would be uploading their own CLKs using their own update token.

These CLKs are already hashed using [clkhash](https://github.com/data61/clkhash), so for each data provider, we just need to upload their corresponding hash file.

In [9]:
for i, token in enumerate(update_tokens, start=1):
    with open(f"data/clks-{i}.json") as f:
        r = requests.post(
            f"{PREFIX}/projects/{project_id}/clks",
            data=f,
            headers={
                "Authorization": token,
                "content-type": "application/json"
            }
        )
    print(f"Data provider {i}: {r.text}")

Data provider 1: {
  "message": "Updated",
  "receipt_token": "795739e4d364fbbc35b9975c7e751f864ef372cd6ede2c5c"
}

Data provider 2: {
  "message": "Updated",
  "receipt_token": "446be8109f13ffb58a87f526c77badb8c705f1cb1cd062cd"
}

Data provider 3: {
  "message": "Updated",
  "receipt_token": "503eeaa96883aaa4dd78ad1671a4ba158f476673564d2783"
}

Data provider 4: {
  "message": "Updated",
  "receipt_token": "3cb842a5cdcfcc19410204cd141dcd90609701ff066f3c73"
}

Data provider 5: {
  "message": "Updated",
  "receipt_token": "49cc67666c200db6db642a18f3dcd74debbca1a4d079d938"
}



## Begin a run
The data providers have uploaded their CLKs, so we may begin the computation. This computation may be repeated multiple times, each time with different parameters. Each such repetition is called a run. The most important parameter to vary between runs is the similarity threshold. Two records whose similarity is above this threshold will be considered to describe the same entity.

Here, we perform one run. We (somewhat arbitrarily) choose the threshold to be 0.8.

In [10]:
r = requests.post(
    f"{PREFIX}/projects/{project_id}/runs",
    headers={
        "Authorization": result_token
    },
    json={
        "threshold": 0.8
    }
)
run_id = r.json()["run_id"]

## Check the status
Let's see whether the run has finished ('state' is 'completed')!

In [18]:
r = requests.get(
    f"{PREFIX}/projects/{project_id}/runs/{run_id}/status",
    headers={
        "Authorization": result_token
    }
)
r.json()

{'current_stage': {'description': 'compute output', 'number': 3},
 'stages': 3,
 'state': 'completed',
 'time_added': '2019-06-17T04:48:35.415224+00:00',
 'time_completed': '2019-06-17T04:48:55.962677+00:00',
 'time_started': '2019-06-17T04:48:36.826288+00:00'}

## Retrieve the results
We retrieve the results of the linkage. As we selected earlier, the result is a list of groups of records. Every record in such a group belongs to the same entity and consists of two values, the party id and the row index.

The last 20 groups look like this.

In [20]:
r = requests.get(
    f"{PREFIX}/projects/{project_id}/runs/{run_id}/result",
    headers={
        "Authorization": result_token
    }
)
groups = r.json()
groups['groups'][-20:]

[[[1, 1884], [2, 1911], [4, 1926], [0, 1916]],
 [[2, 3173], [4, 3176], [3, 3163], [0, 3145], [1, 3161]],
 [[0, 444], [1, 423]],
 [[0, 1206], [4, 1205], [2, 1206]],
 [[1, 797], [3, 823], [2, 833], [0, 813]],
 [[0, 741], [3, 752]],
 [[0, 1902], [4, 1911], [1, 1868], [3, 1895], [2, 1899]],
 [[3, 2833], [4, 2837], [1, 2842]],
 [[2, 1631], [3, 1631]],
 [[0, 143], [1, 142], [4, 121]],
 [[0, 1912], [4, 3137]],
 [[2, 538], [3, 542]],
 [[2, 2196], [3, 2189], [4, 2190], [1, 2188]],
 [[0, 105], [3, 83], [1, 107]],
 [[1, 2526], [2, 2545], [4, 2516]],
 [[3, 1140], [4, 837]],
 [[0, 3175], [1, 3192], [4, 3204]],
 [[3, 1055], [4, 1064], [2, 1074], [1, 1021]],
 [[0, 2009], [2, 2003]],
 [[1, 848], [4, 871], [3, 880]]]

To sanity check, we pick the first 20 groups and print their records' corresponding PII.

In [23]:
def load_dataset(i):
    dataset = []
    with open(f"data/dataset-{i}.csv") as f:
        reader = csv.reader(f)
        next(reader)  # ignore header
        for row in reader:
            dataset.append(row[1:])
    return dataset

datasets = list(map(load_dataset, range(1,6)))

for group in itertools.islice(groups["groups"][-20], 20):
    for i, j in group:
        print(i, datasets[i][j])
    print()

1 ['georgia', 'nguyen', '06-11-1930', 'male', 'perth', '247847.799', '08 6560 4063']
2 ['georia', 'nfuyen', '06-11-1930', 'male', 'perrh', '247847.799', '08 6560 4963']
4 ['geortia', 'nguyea', '06-11-1930', 'male', 'pertb', '247847.798', '08 6560 4063']
0 ['egorgia', 'nguyqn', '06-11-1930', 'male', 'peryh', '247847.799', '08 6460 4963']

2 ['harriyon', 'micyelmor', '21-04-1971', 'male', 'pert1>', '291889.942', '04 5633 5749']
4 ['harri5on', 'micyelkore', '21-04-1971', '', 'pertb', '291880.942', '04 5633 5749']
3 ['hariso17', 'micelmore', '21-04-1971', 'male', 'pertb', '291880.042', '04 5633 5749']
0 ['harrison', 'michelmore', '21-04-1981', 'malw', 'preth', '291880.942', '04 5643 5749']
1 ['harris0n', 'michelmoer', '21-04-1971', '', '', '291880.942', '04 5633 5749']

0 ['joshai', 'browne', '30-10-2904', '', 'melbounfe', '522585.205', '03 7150 7587']
1 ['joshua', 'browne', '30-10-2004', 'female', 'melbourne', '522585.205', '03 7150 7587']

0 ['james', 'lavender', '08-02-2000', 'male', 'c

Despite the high amount of noise in the data, the entity service was able to produce a fairly accurate matching. However, Isabella George and Mia/Talia Galbraith are most likely not an actual match.

We may be able to improve on this results by fine-tuning the hashing schema or by changing the threshold.

## Delete the project

In [24]:
r = requests.delete(
    f"{PREFIX}/projects/{project_id}",
    headers={
        "Authorization": result_token
    }
)
print(r.status_code)

204
